In [1]:
# import sys
# !{sys.executable} -m ensurepip --upgrade
# !{sys.executable} -m pip install numpy

In [2]:
# import sys
# !{sys.executable} -m pip install matplotlib

In [1]:
# Neural Networks from Scratch
# Imports
import numpy as np
import draw_nn

In [2]:
# Dense Layer
class Layer_Dense:
    # Initialize
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons) # x0.01 to reduce init. weight
        self.biases = np.zeros((1, n_neurons))

    # Forward Pass
    def forward(self, inputs):
        self.inputs = inputs
        # Calculate output from inputs, weights, and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass
    def backward(self, dvalues):     # dvalues: gradient passed from next layer
        # Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis= 0, keepdims= True)
        # Gradient on inputs (values)
        self.inputs = np.dot(dvalues, self.weights.T)

In [ ]:
# ReLU Activation
class Activation_ReLU:
    # Forward Pass
    def forward(self, inputs):
        self.inputs = inputs
        # Apply ReLU activation on inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Copy
        self.dinputs = dvalues.copy()

        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0


    
# Softmax Activation
class Activation_Softmax:
    # Forward Pass
    def forward(self, inputs):
        # Get unnormalized probabilities
        exp_values = np.exp(inputs - np.maximum(inputs, axis= 1, keepdims= True))
        # Normalize probabilities
        probabilities = exp_values / sum(exp_values, axis= 1, keepdims= True)
        self.output = probabilities

In [4]:
# Losses
class Loss:
    # Calculate the losses (output - ground_truth)
    def calculate(self, inputs, y):
        sample_losses = self.forward(outputs, y)
        # Average loss
        average_loss = np.mean(sample_losses)
        # Return
        return average_loss

# Categorical crossentropy loss
class Categorical_Crossentropy(Loss): # Inherits 'Loss' class
    # Forward Pass
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        # Clip prediction to avoid divisble by 0
        # Clip both sides to avoid shift to either side
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
        # Probabilities
        # If y_true is categorical label
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]

        # If y_true is one-hot label
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis= 1)

        negative_log_likelihood = -np.log(correct_confidences)
        return negative_log_likelihood